In [0]:
import os
import fnmatch
import cv2
import numpy as np
import string
import time
from keras.layers.recurrent import GRU
from PIL import ImageFont, ImageDraw, Image  
from random import randint
from keras.preprocessing.sequence import pad_sequences
from keras.layers.merge import add, concatenate
from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model,load_model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
!rm -rf ReCo

In [0]:
!wget https://www.robots.ox.ac.uk/~vgg/data/text/mjsynth.tar.gz

--2019-08-09 04:22:40--  https://www.robots.ox.ac.uk/~vgg/data/text/mjsynth.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10678411583 (9.9G) [application/x-gzip]
Saving to: ‘mjsynth.tar.gz’

mjsynth.tar.gz      100%[===================>]   9.94G  31.3MB/s    in 5m 28s  

2019-08-09 04:28:08 (31.1 MB/s) - ‘mjsynth.tar.gz’ saved [10678411583/10678411583]



In [0]:
!pip install patool

     |████████████████████████████████| 81kB 3.1MB/s 


In [0]:
import patoolib

In [0]:
patoolib.extract_archive('mjsynth.tar.gz',outdir='/content')

patool: Extracting mjsynth.tar.gz ...
patool: running /bin/tar --extract -z --file mjsynth.tar.gz --directory /content


<h1>Training On Synthetic Dataset<h1>

In [0]:
#char_list:   'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
# total number of our output classes: len(char_list)
char_list = string.ascii_letters+string.digits
 
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
        
    return dig_lst

In [0]:
path = 'mnt/ramdisk/max/90kDICT32px'
 
# lists for training dataset
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
 
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []
 
max_label_len = 0
 
i =1 
flag = 0
 
for root, dirnames, filenames in os.walk(path):
 
    for f_name in fnmatch.filter(filenames, '*.jpg'):
        # read input image and convert into gray scale image
        c = cv2.imread(os.path.join(root, f_name))
        img = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)
 
        # convert image into shape of (32, 128, 1)
        w, h = img.shape
        if h > 128 or w > 32:
            img = cv2.resize(img, (128,32))
            w,h = img.shape
        if w < 32:
            add_zeros = np.ones((32-w, h))*255
            img = np.concatenate((img, add_zeros))
 
        if h < 128:
            add_zeros = np.ones((32, 128-h))*255
            img = np.concatenate((img, add_zeros), axis=1)
        img = np.expand_dims(img , axis = 2)
        
        # Normalize each image
        img = img/255.
        
        # get the text from the image
        txt = f_name.split('_')[1]
        
        #max len of the text
        if len(txt) > max_label_len:
            max_label_len = len(txt)
            
           
        # split data into validation and training dataset as 10% and 90% respectively
        if i%10 == 0:
            valid_orig_txt.append(txt)   
            valid_label_length.append(len(txt))
            valid_input_length.append(31)
            valid_img.append(img)
            valid_txt.append(encode_to_labels(txt))
        else:
            orig_txt.append(txt)   
            train_label_length.append(len(txt))
            train_input_length.append(31)
            training_img.append(img)
            training_txt.append(encode_to_labels(txt)) 
        
        
        if i == 300000:
            flag = 1
            break
        i+=1
    if flag == 1:
        break

In [0]:
max_label_len

22

In [0]:
len(train_input_length)

270000

In [0]:
# pad each output to maximum text length (22)
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

In [0]:
#Input in Model in shape of number of images, number of rows, number of colums , number of channel 

# input shape row=32 and col=128 channel=1
inputs = Input(shape=(32,128,1))
# Convolution Layer

# 1st Convolution Layer kernel size (3,3)
x = Conv2D(64, (3,3), activation = 'relu', padding='same', name='conv1')(inputs)
x = MaxPool2D(pool_size=(2, 2), strides=2 , name='max1')(x)
 
# 2nd Convolution Layer kernel size (3,3)
x = Conv2D(128, (3,3), activation = 'relu', padding='same' , name='conv2')(x)
x = MaxPool2D(pool_size=(2, 2), strides=2 , name='max2')(x)
 
# 3rd Convolution Layer kernel size (3,3)
x = Conv2D(256, (3,3), activation = 'relu', padding='same',name='conv3')(x)
 
# 4th Convolution Layer kernel size (3,3)
x = Conv2D(256, (3,3), activation = 'relu', padding='same',name='conv4')(x)
x = MaxPool2D(pool_size=(2, 1) , name='max3')(x)
 
# 5th Convolution Layer kernel size (3,3)
x = Conv2D(512, (3,3), activation = 'relu', padding='same',name='conv5')(x)
x = BatchNormalization()(x)
 
# 6th Convolution Layer kernel size (3,3)
x = Conv2D(512, (3,3), activation = 'relu', padding='same',name='conv6')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 1) , name='max4')(x)
 
# 7th Convolution Layer
x = Conv2D(512, (2,2), activation = 'relu',name='conv7')(x)
 
# Making input compatible with BILSTM
squeezed = Lambda(lambda x: K.squeeze(x, 1))(x)
 
# Bidirectional LSTM layers 128 units
bilstm1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2,name='lstm1'))(squeezed)
bilstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2,name='lstm2'))(bilstm1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(bilstm2)

# Test Model
active_model = Model(inputs, outputs)

W0808 13:11:30.522094 139934190012288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 13:11:30.595730 139934190012288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 13:11:30.629390 139934190012288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 13:11:30.672574 139934190012288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0808 13:11:30.748741 139934190012288 deprecation_wrapp

In [0]:
active_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 128, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 128, 64)       640       
_________________________________________________________________
max1 (MaxPooling2D)          (None, 16, 64, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 64, 128)       73856     
_________________________________________________________________
max2 (MaxPooling2D)          (None, 8, 32, 128)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 8, 32, 256)        295168    
_________________________________________________________________
conv4 (Conv2D)               (None, 8, 32, 256)        590080    
__________

In [0]:
max_label_len

22

In [0]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

#model will be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

W0808 13:11:35.713020 139934190012288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4249: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0808 13:11:35.828937 139934190012288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0808 13:11:35.872643 139934190012288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4229: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 128, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 128, 64)  640         input_1[0][0]                    
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 16, 64, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 16, 64, 128)  73856       max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

In [0]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam',metrics=['accuracy'])
 
filepath="txt_recog.weights"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

W0808 13:11:35.912228 139934190012288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
train_input_length[10]

31

In [0]:
training_img = np.array(training_img)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

In [0]:
batch_size = 500
epochs = 11
model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(len(training_img)), batch_size=batch_size, epochs = epochs, validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]), verbose = 1, callbacks = callbacks_list)

Train on 270000 samples, validate on 30000 samples
Epoch 1/11
270000/270000 [==============================] - 764s 3ms/step - loss: 28.4864 - acc: 0.0000e+00 - val_loss: 29.3895 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 29.38946, saving model to txt_recog.weights
Epoch 2/11
270000/270000 [==============================] - 749s 3ms/step - loss: 9.5211 - acc: 0.1166 - val_loss: 4.6124 - val_acc: 0.3541

Epoch 00002: val_loss improved from 29.38946 to 4.61241, saving model to txt_recog.weights
Epoch 3/11
270000/270000 [==============================] - 744s 3ms/step - loss: 4.0290 - acc: 0.4212 - val_loss: 3.7184 - val_acc: 0.5182

Epoch 00003: val_loss improved from 4.61241 to 3.71835, saving model to txt_recog.weights
Epoch 4/11
270000/270000 [==============================] - 739s 3ms/step - loss: 3.1689 - acc: 0.5349 - val_loss: 3.1426 - val_acc: 0.5809

Epoch 00004: val_loss improved from 3.71835 to 3.14263, saving model to txt_recog.weights
Epoch 5/11
270000

In [0]:
c=0
for root, dirnames, filenames in os.walk(path):
 
    for f_name in fnmatch.filter(filenames, '*.jpg'):
      c= c+1  
      
print(c)

8919273


In [0]:
active_model.load_weights('txt_recog.weights')
 
# predicting on validation images
pred = active_model.predict(valid_img[:10])
 
# use CTC decoder
out = K.get_value(K.ctc_decode(pred, input_length=np.ones(pred.shape[0])*pred.shape[1],
                         greedy=True)[0][0])
 
# see the results
i = 0
for x in out:
    print("original_text =  ", valid_orig_txt[i])
    print("predicted_text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')       
    print('\n')
    i+=1

W0808 16:12:21.035959 139934190012288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4303: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


original_text =   KPH
predicted_text = KPH

original_text =   hoicks
predicted_text = hoicks

original_text =   Eclair
predicted_text = Eclair

original_text =   polka
predicted_text = polka

original_text =   LEAKIEST
predicted_text = LEAKIEST

original_text =   Epigram
predicted_text = Epigram

original_text =   Unbolting
predicted_text = Unbolting

original_text =   Mistrals
predicted_text = Mirtrals

original_text =   CREAMIEST
predicted_text = CREAMEST

original_text =   OUTSTRIPPED
predicted_text = OUTSTRIPPED



In [0]:
active_model.save('txt_recog_model.m')

<h1>Fine Tunning in own dataset<h1>

In [0]:
active_model = load_model('txt_recog_model.m')
active_model.load_weights('txt_recog.weights')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
patoolib.extract_archive('ReCo.zip',outdir='/content')

patool: Extracting ReCo.zip ...
patool: running /usr/bin/7z x -o/content -- ReCo.zip
patool: ... ReCo.zip extracted to `/content'.


'/content'

In [0]:
active_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 128, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 128, 64)       640       
_________________________________________________________________
max1 (MaxPooling2D)          (None, 16, 64, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 64, 128)       73856     
_________________________________________________________________
max2 (MaxPooling2D)          (None, 8, 32, 128)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 8, 32, 256)        295168    
_________________________________________________________________
conv4 (Conv2D)               (None, 8, 32, 256)        590080    
__________

In [0]:
for layer in active_model.layers[:-1]:
	layer.trainable = False

In [0]:
active_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 128, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 128, 64)       640       
_________________________________________________________________
max1 (MaxPooling2D)          (None, 16, 64, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 64, 128)       73856     
_________________________________________________________________
max2 (MaxPooling2D)          (None, 8, 32, 128)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 8, 32, 256)        295168    
_________________________________________________________________
conv4 (Conv2D)               (None, 8, 32, 256)        590080    
__________

In [0]:
# lists for training dataset
training_img1 = []
training_txt1 = []
train_input_length1 = []
train_label_length1 = []
orig_txt1 = []
 
#lists for validation dataset
valid_img1 = []
valid_txt1 = []
valid_input_length1 = []
valid_label_length1 = []
valid_orig_txt1 = []
 
max_label_len1 = 0
 
i =1 
flag = 0
path = 'RC/'

count=1

In [0]:
c=0
for root, dirnames, filenames in os.walk(path):
 
    for f_name in fnmatch.filter(filenames, '*.png'):
      c= c+1  
      
print(c)

1919


In [0]:
for root, dirnames, filenames in os.walk(path):
 
    for f_name in fnmatch.filter(filenames, '*.png'):
        # read input image and convert into gray scale image
        count = count +1
        c = cv2.imread(os.path.join(root, f_name))
        img = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)
 
        # convert each image of shape (32, 128, 1)
        w, h = img.shape
        if h > 128 or w > 32:
            img = cv2.resize(img, (128,32))
            w,h = img.shape
        if w < 32:
            add_zeros = np.ones((32-w, h))*255
            img = np.concatenate((img, add_zeros))
        if h < 128:
            add_zeros = np.ones((32, 128-h))*255
            img = np.concatenate((img, add_zeros), axis=1)
        img = np.expand_dims(img , axis = 2)

        # Normalize each image
        img = img/255.
        
        
        txt = f_name.split('.')[0]
        
        # compute maximum length of the text
        if len(txt) > max_label_len1:
            max_label_len1 = len(txt)
            
           
        if count < 1600:
            orig_txt1.append(txt)   
            train_label_length1.append(len(txt))
            train_input_length1.append(31)
            training_img1.append(img)
            training_txt1.append(encode_to_labels(txt))
        else:
            valid_orig_txt1.append(txt)   
            valid_label_length1.append(len(txt))
            valid_input_length1.append(31)
            valid_img1.append(img)
            valid_txt1.append(encode_to_labels(txt))        

_
 
 
 
 
 
 
 
 
-
-
 
 
-
-
-
-
-
-


In [0]:
outputs=active_model.output
inputs=active_model.input

In [0]:
# pad each output label to maximum text length
train_padded_txt1 = pad_sequences(training_txt1, maxlen=max_label_len1, padding='post', value = len(char_list))
valid_padded_txt1 = pad_sequences(valid_txt1, maxlen=max_label_len1, padding='post', value = len(char_list))

In [0]:
labels1 = Input(name='the_labels', shape=[max_label_len1], dtype='float32')
input_length1 = Input(name='input_length', shape=[1], dtype='int64')
label_length1 = Input(name='label_length', shape=[1], dtype='int64')
 
def ctc_lambda_func1(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
loss_out = Lambda(ctc_lambda_func1, output_shape=(1,), name='ctc')([outputs, labels1, input_length1, label_length1])

#model to be used at training time
model = Model(inputs=[inputs, labels1, input_length1, label_length1], outputs=loss_out)

W0808 08:13:09.205605 140005399807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4249: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0808 08:13:09.326371 140005399807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0808 08:13:09.368653 140005399807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4229: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


In [0]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam', metrics=['accuracy'])
 
filepath="text_recog_1.weights"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

W0808 08:13:10.127195 140005399807872 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
training_img1 = np.array(training_img1)
train_input_length1 = np.array(train_input_length1)
train_label_length1 = np.array(train_label_length1)

valid_img1 = np.array(valid_img1)
valid_input_length1 = np.array(valid_input_length1)
valid_label_length1 = np.array(valid_label_length1)

In [0]:
batch_size = 32
epochs = 25
model.fit(x=[training_img1, train_padded_txt1, train_input_length1, 
             train_label_length1],
          y=np.zeros(len(training_img1)), 
          batch_size=batch_size, 
          epochs = epochs, 
          validation_data = ([valid_img1, valid_padded_txt1, valid_input_length1, 
                              valid_label_length1], [np.zeros(len(valid_img1))]), 
          verbose = 1, 
          callbacks = callbacks_list)

Train on 1598 samples, validate on 321 samples
Epoch 1/25
1598/1598 [==============================] - 12s 8ms/step - loss: 70.8152 - acc: 0.0000e+00 - val_loss: 42.8348 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 42.83484, saving model to text_recog_1.weights
Epoch 2/25
1598/1598 [==============================] - 8s 5ms/step - loss: 33.5924 - acc: 0.0000e+00 - val_loss: 26.3061 - val_acc: 0.0000e+00

Epoch 00002: val_loss improved from 42.83484 to 26.30612, saving model to text_recog_1.weights
Epoch 3/25
1598/1598 [==============================] - 9s 5ms/step - loss: 26.6899 - acc: 0.0000e+00 - val_loss: 23.8926 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from 26.30612 to 23.89257, saving model to text_recog_1.weights
Epoch 4/25
1598/1598 [==============================] - 9s 5ms/step - loss: 24.4558 - acc: 0.0000e+00 - val_loss: 22.3834 - val_acc: 0.0000e+00

Epoch 00004: val_loss improved from 23.89257 to 22.38338, saving model to text_recog_1.weigh

In [0]:
len(valid_img1)

208

In [0]:
for layer in active_model.layers[:]:
	layer.trainable = True

In [0]:
# load the saved best model weights
active_model.load_weights('text_recog_1.weights')
 
# predict outputs on validation images
prediction = active_model.predict(valid_img1[40:50])
 
# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 
# see the results
i = 40
for x in out:
    print("orig_txt =  ", valid_orig_txt1[i])
    print("pred_txt = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')       
    print('\n')
    i+=1

W0808 08:17:27.447884 140005399807872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4303: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


orig_txt =   AP280N6507
pred_txt = MAP128DN650727

orig_txt =   WB06H2175
pred_txt = BHB06H2125

orig_txt =   1N4AL2AP3CN501870
pred_txt = 1N4AL2AP31CN501870

orig_txt =   1GCVKREC0EZ197035
pred_txt = 16EVKREM2Z197085

orig_txt =   MH12HV1735
pred_txt = MH02857

orig_txt =   1FMCU9H96DUA80858
pred_txt = 1FMCU2HAP61U440468

orig_txt =   KL41A9786
pred_txt = 1KL2Z1A57288

orig_txt =   2G1FB1ED5B9139757
pred_txt = 2G1FB1ED5B9T839757

orig_txt =   MH12FY7692
pred_txt = S368

orig_txt =   AP10BB3651
pred_txt = AAP40BB3651



In [0]:
active_model.save('text_recog_1.m')